In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pdb
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import scipy 
import itertools
import time
import glob
import pickle
import pandas as pd
from tqdm import tqdm

sys.path.append('/home/akumar/nse/neural_control')
from utils import apply_df_filters, calc_loadings, calc_cascaded_loadings
from loaders import load_sabes
sys.path.append('/home/akumar/nse/neural_control/analysis_scripts/')
from psth_ilmerge import get_top_neurons, get_top_neurons_alt

In [3]:
# Get importance scores for M1
#dframe = '/home/akumar/nse/neural_control/data/indy_decoding_marginal.dat'
dframe_indy = '/mnt/Secondary/data/postprocessed/indy_decoding_df2.dat'
dframe_loco = '/mnt/Secondary/data/postprocessed/loco_decoding_df.dat'

print('Using dframes %s, %s' % (dframe_indy, dframe_loco))

with open(dframe_indy, 'rb') as f:
    rl = pickle.load(f)
indy_df = pd.DataFrame(rl)

with open(dframe_loco, 'rb') as f:
    loco_df = pickle.load(f)
loco_df = pd.DataFrame(loco_df)
loco_df = apply_df_filters(loco_df,
                        loader_args={'bin_width': 50, 'filter_fn': 'none', 'filter_kwargs': {}, 'boxcox': 0.5, 'spike_threshold': 100, 'region': 'M1'},
                        decoder_args={'trainlag': 4, 'testlag': 4, 'decoding_window': 5})
good_loco_files = ['loco_20170210_03.mat',
'loco_20170213_02.mat',
'loco_20170215_02.mat',
'loco_20170227_04.mat',
'loco_20170228_02.mat',
'loco_20170301_05.mat',
'loco_20170302_02.mat']

loco_df = apply_df_filters(loco_df, data_file=good_loco_files)        
dimreduc_df = pd.concat([indy_df, loco_df])

Using dframes /mnt/Secondary/data/postprocessed/indy_decoding_df2.dat, /mnt/Secondary/data/postprocessed/loco_decoding_df.dat


/home/akumar/anaconda3/envs/ncontrol/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 0.24.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/akumar/anaconda3/envs/ncontrol/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 0.24.2 when using version 1.1.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [4]:
bin_width = 50
T = 30
data_path = '/mnt/Secondary/data/sabes'
top_neurons_pca, top_neurons_fca, loadings_pca, loadings_fca = get_top_neurons_alt(dimreduc_df, n=10, data_path=data_path, T=T, bin_width=bin_width)

0it [00:00, ?it/s]

35it [00:00, 46.64it/s]


In [5]:
x1 = np.array(loadings_fca)
x2 = np.array(loadings_pca)

In [6]:
rand = np.random.RandomState(100)
#perm = itertools.permutations(np.arange(x1.size))
# permutation test
n = int(1e5)
spr = []
idx = 0
for idx in range(n):
    p = rand.permutation(x1.size)
    x_ = x2[p]
    spr.append(scipy.stats.spearmanr(x1, x_)[0])
    idx += 1
    if idx % 1000 == 0:
        print(idx)

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000


In [8]:
ref_spr = scipy.stats.spearmanr(x1, x2)[0]
print(scipy.stats.spearmanr(x1, x2)[1])

0.11878747205947963


In [10]:
spr = np.array(spr)
print(spr[np.abs(spr) >= np.abs(ref_spr)].size/spr.size)

0.11896
